In [9]:
#!/usr/bin/env python3
"""
Posture classification & evaluation for data_new/

• 遍歷 data_new/p1~p5
• MediaPipe Pose 推論，計算 score → label
• 混淆矩陣、precision/recall/F1
• 記錄無法偵測骨架 / 讀檔失敗的圖片
"""

import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# ---------- 分類規則 ----------
def score_to_label(score: int) -> str:
    if score in (10, 7):
        return "p1"
    if score == 5:
        return "p2"
    if score == 4:
        return "p3"
    if score == 2:
        return "p4"
    if score == 1:
        return "p5"
    return "unknown"

# ---------- 姿勢角、得分 ----------
def angle_between(v1, v2):
    v1, v2 = np.array(v1), np.array(v2)
    cosang = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(np.clip(cosang, -1.0, 1.0)))

def get_posture_score(theta_raise, theta_elbow):
    if theta_raise >= 170:
        return 10
    if 100 < theta_raise < 170:
        return 7
    if 80 <= theta_raise <= 100:
        return 5 if theta_elbow >= 150 else 4
    if 50 <= theta_raise < 80:
        return 2
    if theta_raise < 50:
        return 1
    return 0  # 未定義

def arm_score(lm, side, mp_pose):
    SH = lm[getattr(mp_pose.PoseLandmark, f"{side}_SHOULDER").value]
    EL = lm[getattr(mp_pose.PoseLandmark, f"{side}_ELBOW").value]
    WR = lm[getattr(mp_pose.PoseLandmark, f"{side}_WRIST").value]
    HIP = lm[getattr(mp_pose.PoseLandmark, f"{side}_HIP").value]

    SH = [SH.x, SH.y, SH.z]
    EL = [EL.x, EL.y, EL.z]
    WR = [WR.x, WR.y, WR.z]
    HIP = [HIP.x, HIP.y, HIP.z]

    theta_raise = angle_between(np.array(EL) - SH, np.array(HIP) - SH)
    theta_elbow = angle_between(np.array(WR) - EL, np.array(SH) - EL)

    return get_posture_score(theta_raise, theta_elbow)



In [11]:
# ---------- 資料夾 ----------
DATA_NEW = Path("../../data_new")
POSE_CLASSES = ["p1", "p2", "p3", "p4", "p5"]

true_labels, pred_labels = [], []
undetected_images, read_failed_images = [], []

In [12]:
# ---------- MediaPipe ----------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)

for cls in POSE_CLASSES:
    for img_path in tqdm(sorted((DATA_NEW/cls).glob("*")), desc=f"Processing {cls}"):
        img = cv2.imread(str(img_path))
        if img is None:
            read_failed_images.append(img_path.relative_to(DATA_NEW).as_posix())
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if not results.pose_landmarks:
            undetected_images.append(img_path.relative_to(DATA_NEW).as_posix())
            continue

        lm = results.pose_landmarks.landmark
        left_score  = arm_score(lm, "LEFT", mp_pose)
        right_score = arm_score(lm, "RIGHT", mp_pose)
        best_score  = max(left_score, right_score)

        true_labels.append(cls)
        pred_labels.append(score_to_label(best_score))

pose.close()

# ---------- 評估 ----------
print("\n📊 混淆矩陣 (順序 p1~p5):")
cm = confusion_matrix(true_labels, pred_labels, labels=POSE_CLASSES)
print(cm)

print("\n📈 分類報告:")
print(classification_report(true_labels, pred_labels,
                            labels=POSE_CLASSES, digits=3))

# ---------- 輸出偵測失敗清單 ----------
all_failed = read_failed_images + undetected_images
if all_failed:
    txt_path = Path("undetected_images.txt")
    with txt_path.open("w", encoding="utf-8") as f:
        f.write("\n".join(all_failed))
    print(f"\n⚠️  共 {len(all_failed)} 張圖片未納入評估，清單已寫入 {txt_path}")
    if read_failed_images:
        print(f"  - cv2 讀取失敗：{len(read_failed_images)} 張")
    if undetected_images:
        print(f"  - Pose 未偵測到人體：{len(undetected_images)} 張")
else:
    print("\n✅ 所有圖片皆成功偵測並納入評估")


Processing p1:   0%|          | 0/100 [00:00<?, ?it/s]

Processing p5: 100%|██████████| 78/78 [02:00<00:00,  1.55s/it]


📊 混淆矩陣 (順序 p1~p5):
[[ 20  26  20   3   2]
 [  0   1   0   0   0]
 [  5   8  19   2   0]
 [ 14  46 194 166 135]
 [  1   2  15  25  29]]

📈 分類報告:
              precision    recall  f1-score   support

          p1      0.500     0.282     0.360        71
          p2      0.012     1.000     0.024         1
          p3      0.077     0.559     0.135        34
          p4      0.847     0.299     0.442       555
          p5      0.175     0.403     0.244        72

    accuracy                          0.321       733
   macro avg      0.322     0.508     0.241       733
weighted avg      0.710     0.321     0.400       733


⚠️  共 128 張圖片未納入評估，清單已寫入 undetected_images.txt
  - cv2 讀取失敗：1 張
  - Pose 未偵測到人體：127 張
